***GENERATED CODE FOR testpipeline1 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import pyspark
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)
        transformationDF = transformationDF.drop('AdmitDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'gender', 'transformation_label': 'String Indexer'}], 'feature': 'gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '412', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'gender'}, {'feature_label': 'gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('gender')
        transformationDF = transformationDF.drop('discharged')
        transformationDF = transformationDF.drop('facid')

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        return df


***AUTOML FUNCTIONS***

In [ ]:
import statsmodels.api as sm


def driverSarima(df):
    df = df.set_index(df.columns[0])
    objSARIMA = sm.tsa.statespace.SARIMAX(
        df, order=[0, 1, 1], seasonal_order=[0, 1, 1, 12])
    model = objSARIMA.fit()
    print(model.summary().tables[0])
    print(model.summary().tables[1])
    modelData = {"model": model, "X_label": list(df.reset_index().columns)[0],
                 "y_label": list(df.reset_index().columns)[1]}
    return modelData


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run testpipeline1Hooks.ipynb
try:
	#sourcePreExecutionHook()

	tetoutputsigo = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/part-00000-cd5c3187-b002-4bb6-a735-f1964e4fa289-c000.csv', 'filename': '1716477333testoutput.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'FilePath': '/TimeSeriesTesting/testoutput.csv', 'viewFileName': 'testoutput.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run testpipeline1Hooks.ipynb
try:
	#transformationPreExecutionHook()

	testpipelinefeatureforecast = TransformationTimeSeriesForecastingMain.run(tetoutputsigo, json.dumps( {"FE": {"functionList": [{"function": "Original"}], "stage_attributes": {}, "featureList": [{"transformationsData": [{"feature_label": "AdmitDate", "transformation_label": "String Indexer"}], "feature": "AdmitDate", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "412", "mean": "", "stddev": "", "min": "01/01/2012", "max": "01/02/2012", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AdmitDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "lengthofstay", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "3.44", "stddev": "2.25", "min": "1.0", "max": "14.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "lengthofstay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rcount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.78", "stddev": "1.34", "min": "0.0", "max": "5.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "rcount"}, {"transformationsData": [{"feature_label": "gender", "transformation_label": "String Indexer"}], "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "412", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dialysisrenalendstage", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.03", "stddev": "0.17", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "dialysisrenalendstage"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "asthma", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.03", "stddev": "0.18", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "asthma"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "irondef", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.08", "stddev": "0.28", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "irondef"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pneum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.02", "stddev": "0.15", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "pneum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "substancedependence", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.07", "stddev": "0.26", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "substancedependence"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "psychologicaldisordermajor", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.19", "stddev": "0.4", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "psychologicaldisordermajo..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "depress", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.03", "stddev": "0.16", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "depress"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "psychother", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.07", "stddev": "0.25", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "psychother"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fibrosisandother", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "fibrosisandother"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "malnutrition", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.05", "stddev": "0.23", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "malnutrition"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hemo", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.06", "stddev": "0.23", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hemo"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hematocrit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "11.54", "stddev": "2.01", "min": "7.0", "max": "20.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hematocrit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "neutrophils", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "10.13", "stddev": "4.32", "min": "2.0", "max": "28.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "neutrophils"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sodium", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "137.51", "stddev": "3.08", "min": "127.0", "max": "147.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "sodium"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "glucose", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "141.39", "stddev": "29.81", "min": "64.0", "max": "224.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "glucose"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bloodureanitro", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "14.7", "stddev": "11.59", "min": "4.0", "max": "159.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bloodureanitro"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creatinine", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "0.65", "stddev": "0.48", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creatinine"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bmi", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "29.4", "stddev": "2.01", "min": "22.0", "max": "35.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bmi"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pulse", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "73.82", "stddev": "10.87", "min": "47.0", "max": "108.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "pulse"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "respiration", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "6.08", "stddev": "0.49", "min": "4.0", "max": "8.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "respiration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "secondarydiagnosisnonicd9", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "412", "mean": "2.32", "stddev": "2.21", "min": "0.0", "max": "10.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "secondarydiagnosisnonicd9"}, {"transformationsData": [{"feature_label": "discharged", "transformation_label": "String Indexer"}], "feature": "discharged", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "412", "mean": "", "stddev": "", "min": "1/15/2012", "max": "9/1/2012", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "discharged"}, {"transformationsData": [{"feature_label": "facid", "transformation_label": "String Indexer"}], "feature": "facid", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "412", "mean": "", "stddev": "", "min": "A", "max": "E", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "facid"}, {"transformationsData": [{}], "feature": "changed date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "changed date"}], "features": {"timecolumn": "changed date", "tocompare": "lengthofstay"}, "dataPercentage": "60", "originalfile": "/FileStore/platform/uploadedSourceFiles/part-00000-cd5c3187-b002-4bb6-a735-f1964e4fa289-c000.csv", "statFunction": {"function": "Original", "parameter": ""}}}))

	#transformationPostExecutionHook(testpipelinefeatureforecast)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run testpipeline1Hooks.ipynb
try:
	#mlPreExecutionHook()

	modelData = driverSarima(testpipelinefeatureforecast)

	#mlPostExecutionHook(modelData)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
from matplotlib import pyplot as plt
try:
    model=modelData['model']
    X_label=modelData['X_label']
    y_label=modelData['y_label']
    plt.clf()
    model.predict(1,264 )
    plt.title('Trending Graphs')
    plt.xlabel(X_label )
    plt.ylabel(y_label )
    plt.grid(color='lightgray')
    display()
except Exception as ex:
    logging.error(ex)

spark.stop()


